In [1]:
# Add relevant Jupyter notebook extensions
%load_ext autoreload
%autoreload 2

In [2]:
# You can double-check your Python path like this...
import sys
print("\n".join(sys.path))

c:\Users\cvest\Claudio\Oxford\3rd Year\B1\b1-coding-practical-mt24\notebooks
C:\Users\cvest\Claudio\Oxford\3rd Year\B1\b1-coding-practical-mt24
c:\Users\cvest\anaconda3\envs\a2e\python311.zip
c:\Users\cvest\anaconda3\envs\a2e\DLLs
c:\Users\cvest\anaconda3\envs\a2e\Lib
c:\Users\cvest\anaconda3\envs\a2e

c:\Users\cvest\anaconda3\envs\a2e\Lib\site-packages
c:\Users\cvest\anaconda3\envs\a2e\Lib\site-packages\win32
c:\Users\cvest\anaconda3\envs\a2e\Lib\site-packages\win32\lib
c:\Users\cvest\anaconda3\envs\a2e\Lib\site-packages\Pythonwin


# Simulate closed-loop
After implementing your control functionality, you can simulate the closed-loop with code that looks something like this...

In [3]:
# Import relevant modules
from uuv_mission import Submarine, Trajectory, Mission, ClosedLoop

sub = Submarine()
# Instantiate your controller (depending on your implementation)
closed_loop = ClosedLoop(sub, controller)
mission = Mission.from_csv("path/to/file") # You must implement this method in the Mission class

trajectory = closed_loop.simulate_with_random_disturbances(mission)
trajectory.plot_completed_mission(mission)

NameError: name 'controller' is not defined